In [83]:
import pandas as pd
import numpy as np
import db_connections
from datetime import datetime, timedelta, date

Bon dia Josep, necessitem que ens feu arribar un informe de la freqüència de agendes d?extraccions en tots els consultoris de ABS Falset i el nombre de analítiques que es fan (mitjana per dia d?extracció).

Es una mica urgent

In [57]:
%%time

d1 = datetime.now().date()
d2 = d1.replace(day=1)
d_last = d2 - timedelta(days=1)
d_first = (d2 - pd.DateOffset(years=1)).date()

dblink='@p6209_prod'

q = f"""
    SELECT
        VISI_USUARI_CIP AS CIP,
        TO_DATE(VISI_DATA_VISITA, 'J') AS DATA_VISITA,
        TO_CHAR(TO_DATE(VISI_HORA_VISITA, 'sssss'), 'hh24:mi') AS HORA_VISITA,
        VISI_CENTRE_CODI_CENTRE AS CODI_CENTRE,
        VISI_CENTRE_CLASSE_CENTRE CLASSE_CENTRE,
        CENT_NOM_CENTRE AS CENTRE,
        VISI_SITUACIO_VISITA AS SITUACIO_VISITA
    FROM VISTB043{dblink} LEFT JOIN PRITB010{dblink} ON VISI_CENTRE_CODI_CENTRE = CENT_CODI_CENTRE AND VISI_CENTRE_CLASSE_CENTRE = CENT_CLASSE_CENTRE
    WHERE VISI_UP='00042'
    AND VISI_SERVEI_CODI_SERVEI='EXTRA'
    AND VISI_DATA_VISITA BETWEEN {pd.to_datetime(d_first).to_julian_date().astype(int)} AND {pd.to_datetime(d1).to_julian_date().astype(int)}
    """

df = db_connections.oracle2pd('ecap', q)

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 14.6 s


In [100]:
%%time

df['SITUACIO_VISITA'] = np.where(df['SITUACIO_VISITA']=='R', 'R', 'N')
df2 = df.groupby(['CENTRE','DATA_VISITA','SITUACIO_VISITA'], as_index=False).agg({'CIP':'nunique'})
df3 = df2.pivot(index=['CENTRE','DATA_VISITA'], columns='SITUACIO_VISITA', values='CIP').reset_index().fillna(0)
df3[['N','R']] = df23[['N','R']].astype(int)
df4 = df3.groupby(['CENTRE'], as_index=False).agg({'DATA_VISITA':'nunique','N':'sum','R':'sum'}).rename(
        {'DATA_VISITA':'Dies agenda extraccions','N':'No presentats','R':'Realitzades'}, axis=1)
df4['Mitjana realitzades per dia'] = round(df4['Realitzades']/df4['Dies agenda extraccions'],2)
df4.sort_values('Mitjana realitzades per dia', ascending=False).to_excel('Extraccions_ABS_Falset_2023_10_01_2024_10_22.xlsx', index=False)

CPU times: user 17.8 ms, sys: 0 ns, total: 17.8 ms
Wall time: 23.5 ms
